In [ ]:
# default_exp core


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from transformers import BertForMaskedLM, BertTokenizer, pipeline


ModuleNotFoundError: No module named 'transformers'

In [ ]:
 #export
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert")
unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [ ]:
assert unmasker('D L I P T S S K L V V [MASK] D T S L Q V K K A F F A L V T') == [{'score': 0.11088384687900543,
  'sequence': 'D L I P T S S K L V V L D T S L Q V K K A F F A L V T',
  'token': 5,
  'token_str': 'L'},
 {'score': 0.0840255618095398,
  'sequence': 'D L I P T S S K L V V S D T S L Q V K K A F F A L V T',
  'token': 10,
  'token_str': 'S'},
 {'score': 0.07328332215547562,
  'sequence': 'D L I P T S S K L V V V D T S L Q V K K A F F A L V T',
  'token': 8,
  'token_str': 'V'},
 {'score': 0.06921827048063278,
  'sequence': 'D L I P T S S K L V V K D T S L Q V K K A F F A L V T',
  'token': 12,
  'token_str': 'K'},
 {'score': 0.0638236328959465,
  'sequence': 'D L I P T S S K L V V I D T S L Q V K K A F F A L V T',
  'token': 11,
  'token_str': 'I'}]

In [ ]:
#export
def spacifySeq(seq):
  return "".join([ aa +" " for aa in seq]).strip()

In [ ]:
assert spacifySeq("MENDEL") == "M E N D E L"